# Online trajectory generation from invariant descriptor


This notebook serves as an example of on-line adaptation of demonstrated trajectories in a shape-preserving way


In [1]:
import numpy as np
import invariants_py.data_handler as dh
import matplotlib.pyplot as plt
import invariants_py.reparameterization as reparam
import scipy.interpolate as ip
from invariants_py.opti_calculate_vector_invariants_position_mf import OCP_calc_pos
from invariants_py.opti_generate_position_from_vector_invariants import OCP_gen_pos
from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Load example trajectory data and reparameterize to arclength


In [2]:
data_location = dh.find_data_path('sine_wave.txt')

# Load data
trajectory,time = dh.read_pose_trajectory_from_txt(data_location)

# Reparameterize
pose,time_profile,arclength,nb_samples,stepsize = reparam.reparameterize_trajectory_arclength(trajectory)
arclength_n = arclength/arclength[-1]
trajectory = pose[:,0:3,3]


Visualize the trajectory

In [3]:
# Plot trajectory
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
ax = plt.axes(projection='3d')
ax.view_init(elev=26, azim=140)
ax.plot(trajectory[:,0],trajectory[:,1],trajectory[:,2],'-')


## Calculate invariant model for demonstrated trajectory


Two steps:
1. first specify optimization problem symbolically
2. calculate invariants given measured trajectory

In [4]:
# specify optimization problem symbolically
FS_calculation_problem = OCP_calc_pos(window_len=nb_samples, bool_unsigned_invariants = False, 
                                           w_pos = 1, w_deriv = (10**-5)*np.array([1.0, 1.0, 1.0]), 
                                           w_abs = (10**-6)*np.array([1.0, 1.0]))

# calculate invariants given measurements
invariants, calculate_trajectory, movingframes = FS_calculation_problem.calculate_invariants_global(trajectory,stepsize)
init_vals_calculate_trajectory = calculate_trajectory
init_vals_movingframes = movingframes



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:    11805
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     5907

Total number of variables............................:     2277
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1818
Total number of inequality c

Visualize calculated trajectory and invariants

In [5]:
# Plot trajectory
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')
ax = plt.axes(projection='3d')
ax.plot(trajectory[:,0],trajectory[:,1],trajectory[:,2],'.-')
ax.plot(calculate_trajectory[:,0],calculate_trajectory[:,1],calculate_trajectory[:,2],'.-')

# Plot invariants
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(10,3))
ax1.plot(arclength_n,invariants[:,0])
ax1.set_title('Velocity [m/-]')
ax1.set_xlabel('s [-]')
ax2.plot(arclength_n,invariants[:,1])
ax2.set_title('Curvature [rad/-]')
ax2.set_xlabel('s [-]')
ax3.plot(arclength_n,invariants[:,2])
ax3.set_title('Torsion [rad/-]')
ax3.set_xlabel('s [-]')



Text(0.5, 0, 's [-]')

In [6]:
movingframes[:,0,0]

array([-0.73548207, -0.74418658, -0.75352654, -0.7641319 , -0.77642628,
       -0.79058614, -0.80652644, -0.8239157 , -0.84222057, -0.86077599,
       -0.87886982, -0.89582805, -0.91108803, -0.92424987, -0.93509938,
       -0.94360064, -0.94986356, -0.95409665, -0.95655521, -0.95749457,
       -0.95713536, -0.95564383, -0.95312469, -0.94962137, -0.94512066,
       -0.93956188, -0.93285074, -0.92487801, -0.91554191, -0.90477233,
       -0.8925502 , -0.87891634, -0.86397105, -0.84786824, -0.83080728,
       -0.81302403, -0.79478132, -0.77635832, -0.75803685, -0.74008771,
       -0.72276034, -0.70627214, -0.69079162, -0.67641737, -0.66316372,
       -0.65095503, -0.63962479, -0.62892186, -0.61852472, -0.60805964,
       -0.59711704, -0.58526521, -0.57206655, -0.55710199, -0.54000027,
       -0.52046347, -0.49828643, -0.47337417, -0.44576091, -0.41562852,
       -0.38331937, -0.34934228, -0.31437197, -0.27923861, -0.24490183,
       -0.21240693, -0.18282607, -0.15718752, -0.13639648, -0.12

In [7]:
import json
initialvalues_frames = np.hstack((movingframes[:,0,0], movingframes[:,1,0], movingframes[:,2,0], movingframes[:,0,1], movingframes[:,1,1], movingframes[:,2,1], movingframes[:,0,2], movingframes[:,1,2], movingframes[:,2,2], calculate_trajectory[:,0], calculate_trajectory[:,1], calculate_trajectory[:,2]))
invariants_model = np.hstack((invariants[:,0], invariants[:,1], invariants[:,2]))
json.dumps({"initial_frames":initialvalues_frames.tolist(), "invariants_model":invariants_model.tolist()})


'{"initial_frames": [-0.7354820707527998, -0.7441865792842562, -0.7535265393482449, -0.7641319042555841, -0.7764262827868026, -0.7905861369433176, -0.8065264382392542, -0.8239157031121543, -0.8422205664582105, -0.8607759889777776, -0.8788698225908399, -0.8958280498089974, -0.9110880312589881, -0.9242498653445215, -0.9350993824612414, -0.9436006426715684, -0.9498635627578491, -0.9540966511611219, -0.956555211689457, -0.9574945745061234, -0.9571353646216326, -0.955643827941104, -0.9531246892185798, -0.9496213650376302, -0.9451206613064747, -0.9395618759506432, -0.9328507416293833, -0.9248780095534326, -0.9155419119944411, -0.9047723326709101, -0.8925501979730056, -0.8789163355088694, -0.8639710529235579, -0.8478682384196513, -0.8308072758187404, -0.8130240286912706, -0.7947813165055523, -0.7763583213359967, -0.7580368473216657, -0.7400877078482362, -0.7227603396939214, -0.7062721447107446, -0.6907916181176871, -0.6764173668623314, -0.6631637239843898, -0.6509550254395495, -0.639624787760

## Trajectory generation during execution

Goal is to generate a single new trajectory during execution towards a new target, assuming we know the current progress s0

In [8]:
# Function that allows to re-sample the invariant model given the progress range [s0,s1] in which we want a trajectory
def interpolate_model_invariants(demo_invariants, progress_values):
    
    resampled_invariants = np.array([demo_invariants(i) for i in progress_values]) 
    new_stepsize = progress_values[1] - progress_values[0] 
    resampled_invariants[:,0] = resampled_invariants[:,0] *  (progress_values[-1] - progress_values[0])
    return resampled_invariants, new_stepsize


### Resample the invariant model for the new progress range [s0,s1]

In [9]:
# Parameterize invariants as a spline
knots = np.concatenate(([arclength_n[0]],[arclength_n[0]],arclength_n,[arclength_n[-1]],[arclength_n[-1]]))
spline_model_trajectory = ip.BSpline(knots,invariants,3)

# Define new start progress s0
current_progress = 0.4
number_samples = 40
progress_values = np.linspace(current_progress, arclength_n[-1], number_samples)
model_invariants,new_stepsize = interpolate_model_invariants(spline_model_trajectory,progress_values)

# Plot invariants
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(10,3))
ax1.plot(arclength_n,invariants[:,0])
ax1.plot(progress_values,model_invariants[:,0],'r.')
ax1.set_title('Velocity [m/m]')
ax2.plot(arclength_n,invariants[:,1])
ax2.plot(progress_values,model_invariants[:,1],'r.')
ax2.set_title('Curvature [rad/m]')
ax3.plot(arclength_n,invariants[:,2])
ax3.plot(progress_values,model_invariants[:,2],'r.')
ax3.set_title('Torsion [rad/m]')

print('Assume the task is at s=0.4, resample invariants between s=0.4 and s=1.0:')


Assume the task is at s=0.4, resample invariants between s=0.4 and s=1.0:


### Define new constraints and solve the optimization problem to retrieve the trajectory

In [10]:
# new constraints
current_index = round(current_progress*len(trajectory))
p_obj_start = calculate_trajectory[current_index]
p_obj_end = calculate_trajectory[-1] - np.array([-0.2, 0.0, 0.0])
R_FS_start = movingframes[current_index]
R_FS_end = movingframes[-1]

# specify optimization problem symbolically
FS_online_generation_problem = OCP_gen_pos(N=number_samples,w_invars = 10**2*np.array([10**1, 1.0, 1.0]))

# Solve
calculate_trajectory = init_vals_calculate_trajectory
movingframes = init_vals_movingframes
new_invars, new_trajectory, new_movingframes = FS_online_generation_problem.generate_trajectory(U_demo = model_invariants, 
                                                                                                p_obj_init = calculate_trajectory, 
                                                                                                R_t_init = movingframes, 
                                                                                                R_t_start = R_FS_start, 
                                                                                                R_t_end = R_FS_end, 
                                                                                                p_obj_start = p_obj_start, 
                                                                                                p_obj_end = p_obj_end, 
                                                                                                step_size = new_stepsize)



This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     3075
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     1305

Total number of variables............................:      597
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      480
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 4.23e-01 0.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

### Visualize resulting trajectory and invariants

In [11]:
# Plot trajectory
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=26, azim=140)
ax.plot(trajectory[:,0],trajectory[:,1],trajectory[:,2],'b')
ax.plot(new_trajectory[:,0],new_trajectory[:,1],new_trajectory[:,2],'r')
ax.plot(new_trajectory[-1,0],new_trajectory[-1,1],new_trajectory[-1,2],'r*')
ax.plot(new_trajectory[0,0],new_trajectory[0,1],new_trajectory[0,2],'ro')

# Plot invariants
fig = plt.figure(figsize=(14,5))
plt.subplot(1,3,1)
plt.plot(progress_values,new_invars[:,0],'r')
plt.plot(arclength_n,invariants[:,0],'b')
plt.plot(0,0)
plt.title('Velocity [m/-]')
plt.subplot(1,3,2)
plt.plot(progress_values,(new_invars[:,1]),'r')
plt.plot(arclength_n,invariants[:,1],'b')
plt.plot(0,0)
plt.title('Curvature [rad/-]')
plt.subplot(1,3,3)
plt.plot(progress_values,(new_invars[:,2]),'r')
plt.plot(arclength_n,invariants[:,2],'b')
plt.plot(0,0)
plt.title('Torsion [rad/-]')
plt.show()


/tmp/ipykernel_142293/2650737203.py:27: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Online trajectory generation

Continuously generate new trajectories during execution towards a changing target

In [12]:
# Specify optimization problem symbolically
window_len = 20
FS_online_generation_problem = OCP_gen_pos(N=window_len,w_invars = 10**1*np.array([10**1, 1.0, 1.0]))

# Initializing loop
current_progress = 0.0
old_progress = 0.0
calculate_trajectory = init_vals_calculate_trajectory
movingframes = init_vals_movingframes

while current_progress <= 1.0:
    
    print(f"current progress = {current_progress}")

    # Resample invariants for current progress
    progress_values = np.linspace(current_progress, arclength_n[-1], window_len)
    model_invariants,new_stepsize = interpolate_model_invariants(spline_model_trajectory,progress_values)
    
    # Select new boundary constraints
    current_index = round( (current_progress - old_progress) * len(calculate_trajectory))
    #print(current_index)
    p_obj_start = calculate_trajectory[current_index]
    p_obj_end = trajectory[-1] - current_progress*np.array([-0.2, 0.0, 0.0])
    R_FS_start = movingframes[current_index] 
    R_FS_end = movingframes[-1] 

    # Calculate remaining trajectory
    new_invars, calculate_trajectory, movingframes = FS_online_generation_problem.generate_trajectory(U_demo = model_invariants, 
                                                                                                      p_obj_init = calculate_trajectory, 
                                                                                                      R_t_init = movingframes, R_t_start = R_FS_start, 
                                                                                                      R_t_end = R_FS_end, p_obj_start = p_obj_start, 
                                                                                                      p_obj_end = p_obj_end, step_size = new_stepsize)

    # Plot trajectory
    clear_output(wait=True)
    fig = plt.figure(figsize=(14,8))
    ax = fig.add_subplot(231, projection='3d')
    ax.view_init(elev=26, azim=140)
    ax.plot(trajectory[:,0],trajectory[:,1],trajectory[:,2],'b')
    ax.plot(calculate_trajectory[:,0],calculate_trajectory[:,1],calculate_trajectory[:,2],'r')
    ax.plot(calculate_trajectory[-1,0],calculate_trajectory[-1,1],calculate_trajectory[-1,2],'r*')
    ax.plot(calculate_trajectory[0,0],calculate_trajectory[0,1],calculate_trajectory[0,2],'ro')
    
    # Plot invariants
    plt.subplot(2,3,4)
    plt.plot(progress_values,new_invars[:,0],'r')
    plt.plot(arclength_n,invariants[:,0],'b')
    plt.plot(0,0)
    plt.title('velocity [m/m]')
    plt.subplot(2,3,5)
    plt.plot(progress_values,(new_invars[:,1]),'r')
    plt.plot(arclength_n,invariants[:,1],'b')
    plt.plot(0,0)
    plt.title('curvature [rad/m]')
    plt.subplot(2,3,6)
    plt.plot(progress_values,(new_invars[:,2]),'r')
    plt.plot(arclength_n,invariants[:,2],'b')
    plt.plot(0,0)
    plt.title('torsion [rad/m]')
    plt.show()
    
    # New progress
    old_progress = current_progress
    current_progress = old_progress + 1/window_len
    
  

current progress = 0.9500000000000003
This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     1515
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      645

Total number of variables............................:      297
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      240
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.42e-02 0.00e+00  

  13  2.6386279e+05 1.62e-08 4.02e-04  -5.7 5.00e-04  -2.7 1.00e+00 1.00e+00h  1
  14  2.6386279e+05 8.88e-16 1.71e-09  -8.6 5.29e-08  -3.1 1.00e+00 1.00e+00h  1

Number of Iterations....: 14

                                   (scaled)                 (unscaled)
Objective...............:   2.6386278700645582e+05    2.6386278700645582e+05
Dual infeasibility......:   1.7114185624222955e-09    1.7114185624222955e-09
Constraint violation....:   8.8817841970012523e-16    8.8817841970012523e-16
Variable bound violation:   0.0000000000000000e+00    0.0000000000000000e+00
Complementarity.........:   0.0000000000000000e+00    0.0000000000000000e+00
Overall NLP error.......:   6.4459001246877352e-13    1.7114185624222955e-09


Number of objective function evaluations             = 45
Number of objective gradient evaluations             = 10
Number of equality constraint evaluations            = 55
Number of inequality constraint evaluations          = 0
Number of equality constraint Jacobian ev

## Future work

implement all existing optimization problems:
- *invariant types*: screw axis invariants and vector invariants
- *measurement data*: poses, screws, vectors
- *problem types*: invariants calculation, trajectory generation


